In [1]:
!pip install numpy matplotlib -q
! pip install -q transformers[torch] einops accelerate bitsandbytes peft trl

import warnings
warnings.filterwarnings("ignore")

ImportError: cannot import name 'SchemaInferenceError' from 'datasets.arrow_writer' (/opt/conda/lib/python3.10/site-packages/datasets/arrow_writer.py)

In [2]:
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import pipeline
import re

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import os
import json
import glob

files=[]

for fi in glob.glob("/kaggle/input/finetune/*.txt"):
    files.append(fi)

files

['/kaggle/input/finetune/3.txt',
 '/kaggle/input/finetune/10.txt',
 '/kaggle/input/finetune/19.txt',
 '/kaggle/input/finetune/5.txt',
 '/kaggle/input/finetune/7.txt',
 '/kaggle/input/finetune/8.txt',
 '/kaggle/input/finetune/18.txt',
 '/kaggle/input/finetune/17.txt',
 '/kaggle/input/finetune/11.txt',
 '/kaggle/input/finetune/0.txt',
 '/kaggle/input/finetune/9.txt',
 '/kaggle/input/finetune/16.txt',
 '/kaggle/input/finetune/1.txt',
 '/kaggle/input/finetune/13.txt',
 '/kaggle/input/finetune/14.txt',
 '/kaggle/input/finetune/12.txt',
 '/kaggle/input/finetune/15.txt',
 '/kaggle/input/finetune/2.txt',
 '/kaggle/input/finetune/4.txt',
 '/kaggle/input/finetune/6.txt']

In [5]:
from datasets import load_dataset
dataset = load_dataset("text", data_files=files)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-46ee42a2ecad3455/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
model="meta-llama/Llama-2-7b-hf"
tokenizer=AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=64, truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=2, remove_columns=["text"])
    
def copy_input_ids(example):
    example["labels"] = example["input_ids"].copy()
    return example

tokenized_datasets = tokenized_datasets.map(copy_input_ids)

#0:   0%|          | 0/15 [00:00<?, ?ba/s]

#1:   0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/29310 [00:00<?, ?ex/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="pt")

In [11]:
from transformers import BitsAndBytesConfig

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [14]:
original_model = AutoModelForCausalLM.from_pretrained(
        model,
        quantization_config=bnb_config,
        device_map="auto"
    )

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [15]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=16,
    lora_dropout=0.1,

    bias="none",
    task_type="CAUSAL_LM"
)

In [16]:
peft_model = get_peft_model(original_model,lora_config)

In [20]:
from transformers import TrainingArguments, Trainer

In [26]:
peft_training_args = TrainingArguments(
    output_dir="llama-2-7b-test1",
    learning_rate=1e-3,
    num_train_epochs=1,
    weight_decay=0.001,
    per_device_train_batch_size=4,
    save_strategy="steps",
    #evaluation_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    #load_best_model_at_end=True,
    fp16=True,
    report_to='none'
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],
    tokenizer=tokenizer
)

In [27]:
peft_trainer.train()

Step,Training Loss
500,1.343000
1000,1.356100
1500,1.316500
2000,1.285800
2500,1.263000
3000,1.236200
3500,1.226300
4000,1.195200
4500,1.172500
5000,1.163800


TrainOutput(global_step=7328, training_loss=1.2153736264424553, metrics={'train_runtime': 11568.6312, 'train_samples_per_second': 2.534, 'train_steps_per_second': 0.633, 'total_flos': 7.44131237511168e+16, 'train_loss': 1.2153736264424553, 'epoch': 1.0})